In [1]:
!pip install gradio --upgrade
!pip install folium --upgrade
!pip install geopy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 755.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

In [6]:
import gradio as gr
import folium
import pandas as pd
import pickle
from datetime import datetime
from geopy.geocoders import Nominatim
from google.colab import drive
from geopy.point import Point
from pyproj import Proj, Transformer

drive.mount('/content/drive')

parking_info_df = pd.read_csv('/content/drive/My Drive/NUS-ISS AIS Projects/Project 2/Data/HDBCarparkInformation.csv')

# Load KNN model
#with open('knn_model_v2.pkl', 'rb') as f:
    #knn_model = pickle.load(f)

def get_singapore_locations():
    geolocator = Nominatim(user_agent="best_parking_location_finder_app")
    location = geolocator.geocode("Singapore")

    # Get the bounding box for Singapore
    south, north, west, east = location.raw['boundingbox']

    # Use the bounding box to query for places within Singapore
    places = geolocator.geocode(query="*", viewbox=[Point(south, west), Point(north, east)], bounded=1)

    if places is None:
        print("No places found within the bounding box.")
        return []  # Return an empty list if places is None

    # Extract and return the location names
    location_names = [place.address for place in places if place]
    return location_names

locations = get_singapore_locations()  # Store the list of locations

locations = parking_info_df['address'].unique().tolist()  #Just in case, geolocator doesn't work we can use this to get input locations

parking_info_df = parking_info_df.rename(columns={'car_park_no': 'car_park_number'})
parking_info_df = parking_info_df.drop('gantry_height', axis=1, errors='ignore')

# Define a function to get the best parking location
def get_best_parking_location(input_latitude, input_longitude, input_time, car_park_type, free_parking, car_park_basement, night_parking):

    input_latitude, input_longitude, input_address = get_location_from_map(map_data)
    # Convert the input time to the required format (if needed)
    # Process location and time as model inputs
    #model_input = [input_location, input_time]  # Modify as necessary for model input format
    #prediction = knn_model.predict([model_input])

    #randomly select 10 car park ID
    prediction = parking_info_df['carpark_number'].sample(n=10).tolist()  #Randomly picked 3 10 locations to check working of UI

    print(prediction)

    #Get address of the predictions using parking_info_df
    address_list = parking_info_df[parking_info_df['carpark_number'].isin(prediction)]['address'].tolist()
    print(address_list)

    recommendations = []
    for parking_id in prediction:
        parking_details = parking_info_df[parking_info_df['carpark_number'] == parking_id].iloc[0].to_dict()  # Convert to dictionary
        details_string = f"Location: {parking_details['address']}, Type: {parking_details['car_park_type']}, Parking System: {parking_details['type_of_parking_system']},  Free Parking: {parking_details['free_parking']},  Night Parking: {parking_details['night_parking']},  Car Park Decks: {parking_details['car_park_decks']},  Basement Parking: {parking_details['car_park_basement']}"  # Format details
        recommendations.append(details_string)

        if (car_park_type == "Any" or parking_details['car_park_type'] == car_park_type) and \
           (free_parking == "Any" or parking_details['free_parking'] == free_parking) and \
           (car_park_basement == "Any" or parking_details['car_park_basement'] == car_park_basement) and \
           (night_parking == "Any" or parking_details['night_parking'] == night_parking):
            recommendations.append(parking_id)
            recommendations.append(details_string)

    # Get top 3 filtered recommendations
    top_3_recommendations = recommendations[:3]

    return ", ".join(recommendations)  # Convert list to comma-separated string

def convert_lat_long_to_xy(latitude, longitude):
    wgs84_proj = Proj(proj="latlong", datum="WGS84")
    svy21_proj = Proj("epsg:3414")

    transformer = Transformer.from_proj(wgs84_proj, svy21_proj)
    x, y = transformer.transform(longitude, latitude)

    return x, y

def create_map():
    m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)  # Centered on Singapore
    m.add_child(folium.LatLngPopup())  # Allows clicking to get coordinates
    return m

def get_location_from_map(map_data):
    # Extract latitude and longitude from map_data
    if map_data:
        lat, lng = map_data
        # (Optional) Reverse geocode to get address
        geolocator = Nominatim(user_agent="best_parking_location_finder_app")
        location = geolocator.reverse(f"{lat}, {lng}")
        address = location.address if location else "Address not found"
        print(f"Latitude: {lat}, Longitude: {lng}, Address: {address}")

        #Converting lat long into x, y
        x, y = convert_lat_long_to_xy(lat, lng)
        print(f"X: {x}, Y: {y}")

        return x, y, address
    else:
        return None, None, None

map_component = gr.HTML(lambda: create_map()._repr_html_()) # Using gr.HTML to display ma


# Create Gradio interface
#location_dropdown = gr.Dropdown(choices=locations, label="Select or Type Location", interactive=True)
time_input = gr.Textbox(label="Enter Time (HH:MM)", placeholder="e.g., 14:30")
car_park_type_dropdown = gr.Dropdown(choices=["Any", "Surface Car Park", "Multi-Storey Car Park"], label="Car Park Type")
free_parking_dropdown = gr.Dropdown(choices=["Any", "Yes", "No"], label="Free Parking")
car_park_basement_dropdown = gr.Dropdown(choices=["Any", "Yes", "No"], label="Basement Parking")
night_parking_dropdown = gr.Dropdown(choices=["Any", "Yes", "No"], label="Night Parking")
output_text = gr.Textbox(label="Best Parking Locations")

# Launch the Gradio interface

iface = gr.Interface(
    fn=get_best_parking_location,
    inputs=[map_component, time_input, car_park_type_dropdown, free_parking_dropdown, car_park_basement_dropdown, night_parking_dropdown],
    outputs=output_text,
    title="Best Parking Location Finder",
    description="Click on the map to select a location and enter a time to find the best parking option.",
)
iface.launch()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No places found within the bounding box.


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:980: UserWarning: Expected 7 arguments for function <function get_best_parking_location at 0x7e0590c7ea70>, received 6.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:984: UserWarning: Expected at least 7 arguments for function <function get_best_parking_location at 0x7e0590c7ea70>, received 6.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://767ecbdae6c400f333.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
